# From Genotype data to IMI, Breakpts, and Haplotypes

###### From Genotype data calculate for each sample, the intermarker interval (IMI), find genotype changes or break points (breakpts), and construct haplotypes. Then filter this data to generate new genotype data for each sample and visualize the effect of our filter. 

#### Import needed moduals

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import cryptofxns
import os
from matplotlib import pyplot as plt
%matplotlib inline

## Set local variables 

In [2]:
mod = .1;s = 0;ymod = 1;yla = -.5;ylb = 1.5;
xlim0 = -100000;xlim1 = 500000;xdi = 4.0;
k1 = 6000;Method = 'min'
alvl =.3;centcolor='red';snpcolor='black';

### Load Genotype files, centromere locations, and chromosome length

In [3]:
genodf = cryptofxns.geno_df;genodf.shape
centromere = cryptofxns.load_centrom_locs()
clens = cryptofxns.chromosome_lengths
tc = cryptofxns.thecol

### Find Chromosome and gather samples

In [6]:
chrlist = np.unique(genodf.Chrom).tolist()
segs_parents = genodf.columns.tolist()[tc:];
segs_drop = cryptofxns.geno_remove;
segs = [s for s in segs_parents if s not in segs_drop]
len(segs)

97

In [7]:
len(segs_parents)

108

In [8]:
segs_parents

['SS-A837',
 'SS-B307',
 'SS-B308',
 'SS-B310',
 'SS-B311',
 'SS-B312',
 'SS-B313',
 'SS-B314',
 'SS-B315',
 'SS-B316',
 'SS-B317',
 'SS-B350',
 'SS-B354',
 'SS-B356',
 'SS-B357',
 'SS-B358',
 'SS-B360',
 'SS-B364',
 'SS-B366',
 'SS-B368',
 'SS-B369',
 'SS-B373',
 'SS-B374',
 'SS-B377',
 'SS-B378',
 'SS-B379',
 'SS-B382',
 'SS-B383',
 'SS-B385',
 'SS-B387',
 'SS-B395',
 'SS-B396',
 'SS-B397',
 'SS-B399',
 'SS-B407',
 'SS-B410',
 'SS-B411',
 'SS-B553',
 'SS-B560',
 'SS-B563',
 'SS-B564',
 'SS-B565',
 'SS-B569',
 'SS-B570',
 'SS-B574',
 'SS-B575',
 'SS-B579',
 'SS-B580',
 'SS-B584',
 'SS-B585',
 'SS-B586',
 'SS-B587',
 'SS-B588',
 'SS-B589',
 'SS-B591',
 'SS-B593',
 'SS-B594',
 'SS-B598',
 'SS-B599',
 'SS-B600',
 'SS-B605',
 'SS-B606',
 'SS-B611',
 'SS-B621',
 'SS-B869_cor',
 'SS-B872_cor',
 'SS-B873_cor',
 'SS-B874_cor',
 'SS-B876',
 'SS-B879',
 'SS-B880',
 'SS-B885',
 'SS-B886_cor',
 'SS-B887_cor',
 'SS-B890_cor',
 'SS-B892_cor',
 'SS-B896_cor',
 'SS-B898_cor',
 'SS-B901_cor',
 'SS-B90

In [7]:
segs_drop

['SS-B377',
 'SS-B385',
 'SS-B410',
 'SS-B411',
 'SS-B560',
 'SS-B395',
 'SS-B594',
 'SS-C273',
 'SS-B879',
 'SS-B885',
 'SS-B593']

In [6]:
segs_parents

['SS-A837',
 'SS-B307',
 'SS-B308',
 'SS-B310',
 'SS-B311',
 'SS-B312',
 'SS-B313',
 'SS-B314',
 'SS-B315',
 'SS-B316',
 'SS-B317',
 'SS-B350',
 'SS-B354',
 'SS-B356',
 'SS-B357',
 'SS-B358',
 'SS-B360',
 'SS-B364',
 'SS-B366',
 'SS-B368',
 'SS-B369',
 'SS-B373',
 'SS-B374',
 'SS-B377',
 'SS-B378',
 'SS-B379',
 'SS-B382',
 'SS-B383',
 'SS-B385',
 'SS-B387',
 'SS-B395',
 'SS-B396',
 'SS-B397',
 'SS-B399',
 'SS-B407',
 'SS-B410',
 'SS-B411',
 'SS-B553',
 'SS-B560',
 'SS-B563',
 'SS-B564',
 'SS-B565',
 'SS-B569',
 'SS-B570',
 'SS-B574',
 'SS-B575',
 'SS-B579',
 'SS-B580',
 'SS-B584',
 'SS-B585',
 'SS-B586',
 'SS-B587',
 'SS-B588',
 'SS-B589',
 'SS-B591',
 'SS-B593',
 'SS-B594',
 'SS-B598',
 'SS-B599',
 'SS-B600',
 'SS-B605',
 'SS-B606',
 'SS-B611',
 'SS-B621',
 'SS-B869_cor',
 'SS-B872_cor',
 'SS-B873_cor',
 'SS-B874_cor',
 'SS-B876',
 'SS-B879',
 'SS-B880',
 'SS-B885',
 'SS-B886_cor',
 'SS-B887_cor',
 'SS-B890_cor',
 'SS-B892_cor',
 'SS-B896_cor',
 'SS-B898_cor',
 'SS-B901_cor',
 'SS-B90

In [5]:
segs = segs_drop

In [6]:
segs

['SS-B377',
 'SS-B385',
 'SS-B410',
 'SS-B411',
 'SS-B560',
 'SS-B395',
 'SS-B594',
 'SS-C273',
 'SS-B879',
 'SS-B885',
 'SS-B593']

## PATCH!!! April 3rd

### Set folder paths

In [7]:
folders = ['IMI/','BPS/','HAP/','FGT/','FIG/']

CHRPATHS = ['../FILES/CHR' + str(i) +'/' for i in range(1,len(clens)+1)];CHRPATHS[0]

FILECHRPATHS = [ [chp + f for f in folders] for chp in CHRPATHS];FILECHRPATHS[0][:]

['../FILES/CHR1/IMI/',
 '../FILES/CHR1/BPS/',
 '../FILES/CHR1/HAP/',
 '../FILES/CHR1/FGT/',
 '../FILES/CHR1/FIG/']

#### Make directory

In [8]:
for c in range(len(clens)):
    for directory in FILECHRPATHS[c]:
        for seg in segs:
            director = directory+seg
            if not os.path.exists(director):
                os.makedirs(director)

In [9]:
director

'../FILES/CHR14/FIG/SS-B593'

## Calculate IMI, Recombination Breakpts, and Haplotypes

### Patch May 15th

In [10]:
genodf.shape

(96337, 112)

In [11]:
genodf = genodf[genodf.columns.tolist()[:tc]+segs]#.dropna(how='any',axis=0);
genodf.shape

(96337, 15)

In [13]:
for ch,chrom in enumerate(chrlist):
    clen = clens[ch]; 
    geno_ch = genodf[genodf.Chrom==chrom]
    for s,seg in enumerate(segs):
        sub = geno_ch[['Pos',seg]].dropna(axis=0,how='any').reset_index(drop=True); ## Drop empty markers
        if len(sub[seg]) == 0:
            continue
        else:
            #print sub
            subpos = sub.Pos.tolist();
            subgt = sub[seg].tolist();
        ## make IMI dataframe 
            subimi = pd.DataFrame();
            subimi['v'] = subpos[:-1];
            subimi['w'] = subpos[1:];
            subimi['X'] = abs(np.array(subgt[:-1]) - np.array(subgt[1:]));
            subimi['M'] = [np.mean((subimi.w[i],subimi.v[i])) for i in subimi.index.tolist()]; ## Midpoint Method
            subimi.to_csv(FILECHRPATHS[ch][0] +  seg + '/' + seg + '-IMI' + '.csv',
                      index=False);
        
        ## Find break pts dataframe. 
            bp = subimi[subimi.X != 0].dropna(axis=0,how='any');
            bp.to_csv(FILECHRPATHS[ch][1]+ seg + '/' + seg  + '-breakpt' + '.csv',
                  index=False);
        
        ## Make haplotype dataframe. 
            hapdf = cryptofxns.haplotypes(subimi,clen,method=Method);
            hapdf['genotype'] = [ss.mode(sub[seg][(sub.Pos >= hapdf.v[i]) & 
                                              (sub.Pos <= hapdf.w[i])])[0].tolist()[0] 
                             for i in hapdf.index.tolist()]
            hapdf.to_csv(FILECHRPATHS[ch][2] +  seg + '/' + seg +'-'+ Method +'-haplotype' +'-k%s'%0+ '.csv',
                     index=False);
        
        ## Filter Haplotype dataframe. 
            hap = cryptofxns.haplotype_filter(hapdf,k1);
            hap.to_csv(FILECHRPATHS[ch][2]+ seg + '/' + seg +'-'+ Method +'-haplotype' +'-k%s'%k1+ '.csv',
                   index=False);
        
        ## Generate filtered chromosomal positions and genotypes
            tempnewpos = [[pos for pos in subpos if pos <= hap.w[i] and 
                       pos >= hap.v[i]] for i in hap.index.tolist()]
            newgt = np.concatenate([[hap.genotype[K] for p in tempnewpos[K]] 
                                for K in hap.index.tolist()]);
            newpos = np.concatenate(tempnewpos);assert len(newgt) == len(newpos);
        
        ## Save the filtered genotype data
            newseg = pd.DataFrame(columns=['Pos',seg]);
            newseg.Pos = newpos;newseg[seg] = newgt;
            newseg.to_csv(FILECHRPATHS[ch][3]+ seg + '/' + seg +'-'+ Method + '-Geno' + '-k%s'%k1+ '.csv',
                     index=False)
        
        ## Vizualize Haplotypes
            plt.plot(subpos,np.array(subgt)*mod,'.',color=snpcolor,alpha=alvl);
            plt.plot(newpos,newgt*mod + ymod,'.',color=snpcolor,alpha=alvl);
            plt.annotate('Raw: %s'%len(hapdf),xy=(clen+(xlim1/xdi),0));
            plt.annotate('Filterd: %s'%len(hap),xy=(clen+(xlim1/xdi),ymod));
            plt.broken_barh([(centromere[0][chrom],
                          centromere[1][chrom]-centromere[0][chrom])],(ymod,mod),
                        facecolors=centcolor,alpha =alvl);
            plt.broken_barh([(centromere[0][chrom],
                          centromere[1][chrom]-centromere[0][chrom])],(0,mod),
                        facecolors=centcolor,alpha =alvl);
            plt.ylim(yla,ylb);
            plt.xlim(xlim0,clen+xlim1);
            plt.title(seg + ' ' +chrom+'-haplotype ' + '\nMethod: '+ Method +'\nk1: ' + '%s'%k1);
            plt.xlabel('Chromosomal Coordinates');
            plt.ylabel('Genotype');
            cur_axes = plt.gca();
            cur_axes.axes.get_yaxis().set_visible(False);
            plt.tight_layout();
            plt.savefig(FILECHRPATHS[ch][4] + seg + '/'+ seg + '-' +chrom+'-hap-'+ Method +'-k1-%s'%k1+'.jpeg');
            plt.clf();

## Make haplotypes for other filter sizes

In [ ]:
K = np.arange(1000,11500,500).tolist()

In [ ]:
for ch,chrom in enumerate(chrlist):
    clen = clens[ch];
    geno_ch = genodf[genodf.Chrom==chrom]
    for s,seg in enumerate(segs):
        sub = geno_ch[['Pos',seg]].dropna(axis=0,how='any').reset_index(drop=True); ## Drop empty markers
        subpos = sub.Pos.tolist();
        subgt = sub[seg].tolist();
        if len(subgt) == 0:
            continue
        ## Bring in imi dataframe
        subimi = pd.read_csv(FILECHRPATHS[ch][0] +  seg + '/' + seg + '-IMI' + '.csv')
        ## For each filter size, make haplotype dataframe. 
        hapdf = cryptofxns.haplotypes(subimi,clen,method=Method);
        hapdf['genotype'] = [ss.mode(sub[seg][(sub.Pos >= hapdf.v[i]) & 
                                              (sub.Pos <= hapdf.w[i])])[0].tolist()[0] 
                             for i in hapdf.index.tolist()]
        ## Filter Haplotype dataframe. 
        for k1 in K:
            hap = cryptofxns.haplotype_filter(hapdf,k1);
            hap.to_csv(FILECHRPATHS[ch][2]+ seg + '/' + seg +'-'+ Method +'-haplotype' +'-k%s'%k1+ '.csv',
                   index=False);    